In [1]:
from datasets import load_dataset

ds = load_dataset("OpenAssistant/oasst1")

In [2]:
def filter_english(example):
    return example["lang"] == "en"  # Keep only English text

# Apply filtering to all splits
ds_en = {split: ds[split].filter(filter_english) for split in ds.keys()}

# Check the filtered dataset
print(ds_en["train"])
print(ds_en["validation"])


Dataset({
    features: ['message_id', 'parent_id', 'user_id', 'created_date', 'text', 'role', 'lang', 'review_count', 'review_result', 'deleted', 'rank', 'synthetic', 'model_name', 'detoxify', 'message_tree_id', 'tree_state', 'emojis', 'labels'],
    num_rows: 39283
})
Dataset({
    features: ['message_id', 'parent_id', 'user_id', 'created_date', 'text', 'role', 'lang', 'review_count', 'review_result', 'deleted', 'rank', 'synthetic', 'model_name', 'detoxify', 'message_tree_id', 'tree_state', 'emojis', 'labels'],
    num_rows: 2022
})


In [3]:
# Select only text-related fields
fields_to_keep = ["message_id", "parent_id", "text", "role"]

ds_en_clean = {split: ds_en[split].remove_columns(
    [col for col in ds_en[split].column_names if col not in fields_to_keep]
) for split in ds_en.keys()}

# Display a sample
print(ds_en_clean["train"][0:2])


{'message_id': ['6ab24d72-0181-4594-a9cd-deaf170242fb', 'c8e83833-ecbc-44fe-b6db-735228c25a1c'], 'parent_id': [None, '6ab24d72-0181-4594-a9cd-deaf170242fb'], 'text': ['Can you write a short introduction about the relevance of the term "monopsony" in economics? Please use examples related to potential monopsonies in the labour market and cite relevant research.', '"Monopsony" refers to a market structure where there is only one buyer for a particular good or service. In economics, this term is particularly relevant in the labor market, where a monopsony employer has significant power over the wages and working conditions of their employees. The presence of a monopsony can result in lower wages and reduced employment opportunities for workers, as the employer has little incentive to increase wages or provide better working conditions.\n\nRecent research has identified potential monopsonies in industries such as retail and fast food, where a few large companies control a significant porti

In [4]:
import pandas as pd

# Convert the dataset to a list of dictionaries, then to a DataFrame
df_train = ds_en_clean["train"]
df_validation = ds_en_clean["validation"]

# Save to CSV
df_train.to_csv("OpenAssistant_English_Train.csv", index=False)
df_validation.to_csv("OpenAssistant_English_Validation.csv", index=False)

print("CSV files saved successfully!")


Creating CSV from Arrow format:   0%|          | 0/40 [00:00<?, ?ba/s]

Creating CSV from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

CSV files saved successfully!


In [24]:
import pandas as pd

# Load the CSV files
df_train = pd.read_csv("OpenAssistant_English_Train.csv")
df_validation = pd.read_csv("OpenAssistant_English_Validation.csv")

# Inspect the data
print(df_train.head())
print(df_validation.head())

                             message_id                             parent_id  \
0  6ab24d72-0181-4594-a9cd-deaf170242fb                                   NaN   
1  c8e83833-ecbc-44fe-b6db-735228c25a1c  6ab24d72-0181-4594-a9cd-deaf170242fb   
2  6708c47f-05c9-4346-b3d2-40b2bd24fde4  c8e83833-ecbc-44fe-b6db-735228c25a1c   
3  343ee2d4-87ae-41fd-a768-bdd65959dc4a  6ab24d72-0181-4594-a9cd-deaf170242fb   
4  18145bf4-37fd-4ac0-80f5-6108b5f2b365  343ee2d4-87ae-41fd-a768-bdd65959dc4a   

                                                text       role  
0  Can you write a short introduction about the r...   prompter  
1  "Monopsony" refers to a market structure where...  assistant  
2                            Now explain it to a dog   prompter  
3  Monopsony is a market structure in which there...  assistant  
4  How can one fight back when a monospony had be...   prompter  
                             message_id                             parent_id  \
0  e0cf4fa2-e944-44fb-b502-11f155062

In [25]:
import re

# Extract sentence-level features
def extract_style_features(text):
    # Sentence length (number of words)
    words = text.split()
    sentence_length = len(words)

    # Punctuation usage
    question_marks = text.count("?")
    exclamation_marks = text.count("!")

    # Vocabulary richness (unique words)
    unique_words = len(set(words))
    vocabulary_richness = unique_words / len(words) if len(words) > 0 else 0

    return {
        "sentence_length": sentence_length,
        "question_marks": question_marks,
        "exclamation_marks": exclamation_marks,
        "vocabulary_richness": vocabulary_richness,
    }

# Apply to all messages
df_train["style_features"] = df_train["text"].apply(extract_style_features)

In [26]:
pip install torch torchvision#%%
from datasets import load_dataset

ds = load_dataset("OpenAssistant/oasst1")

SyntaxError: invalid syntax (596542121.py, line 1)

In [27]:
def filter_english(example):
    return example["lang"] == "en"  # Keep only English text

# Apply filtering to all splits
ds_en = {split: ds[split].filter(filter_english) for split in ds.keys()}

# Check the filtered dataset
print(ds_en["train"])
print(ds_en["validation"])


NameError: name 'ds' is not defined

In [28]:
# Select only text-related fields
fields_to_keep = ["message_id", "parent_id", "text", "role"]

ds_en_clean = {split: ds_en[split].remove_columns(
    [col for col in ds_en[split].column_names if col not in fields_to_keep]
) for split in ds_en.keys()}

# Display a sample
print(ds_en_clean["train"][0:2])


NameError: name 'ds_en' is not defined

In [ ]:
import pandas as pd

# Convert the dataset to a list of dictionaries, then to a DataFrame
df_train = ds_en_clean["train"]
df_validation = ds_en_clean["validation"]

# Save to CSV
df_train.to_csv("OpenAssistant_English_Train.csv", index=False)
df_validation.to_csv("OpenAssistant_English_Validation.csv", index=False)

print("CSV files saved successfully!")


In [29]:
import pandas as pd

# Load the CSV files
df_train = pd.read_csv("OpenAssistant_English_Train.csv")
df_validation = pd.read_csv("OpenAssistant_English_Validation.csv")

# Inspect the data
print(df_train.head())

                             message_id                             parent_id  \
0  6ab24d72-0181-4594-a9cd-deaf170242fb                                   NaN   
1  c8e83833-ecbc-44fe-b6db-735228c25a1c  6ab24d72-0181-4594-a9cd-deaf170242fb   
2  6708c47f-05c9-4346-b3d2-40b2bd24fde4  c8e83833-ecbc-44fe-b6db-735228c25a1c   
3  343ee2d4-87ae-41fd-a768-bdd65959dc4a  6ab24d72-0181-4594-a9cd-deaf170242fb   
4  18145bf4-37fd-4ac0-80f5-6108b5f2b365  343ee2d4-87ae-41fd-a768-bdd65959dc4a   

                                                text       role  
0  Can you write a short introduction about the r...   prompter  
1  "Monopsony" refers to a market structure where...  assistant  
2                            Now explain it to a dog   prompter  
3  Monopsony is a market structure in which there...  assistant  
4  How can one fight back when a monospony had be...   prompter  


In [30]:
import re

# Extract sentence-level features
def extract_style_features(text):
    # Sentence length (number of words)
    words = text.split()
    sentence_length = len(words)

    # Punctuation usage
    question_marks = text.count("?")
    exclamation_marks = text.count("!")

    # Vocabulary richness (unique words)
    unique_words = len(set(words))
    vocabulary_richness = unique_words / len(words) if len(words) > 0 else 0

    return {
        "sentence_length": sentence_length,
        "question_marks": question_marks,
        "exclamation_marks": exclamation_marks,
        "vocabulary_richness": vocabulary_richness,
    }

# Apply to all messages
df_train["style_features"] = df_train["text"].apply(extract_style_features)

In [31]:
import networkx as nx

# Initialize a directed graph
dialogue_graph = nx.DiGraph()

# Add nodes and edges
for idx, row in df_train.iterrows():
    message_id = row["message_id"]
    parent_id = row["parent_id"]
    text = row["text"]
    role = row["role"]

    # Add node with attributes
    dialogue_graph.add_node(message_id, text=text, role=role)

    # Add edge if parent_id exists
    if parent_id:
        dialogue_graph.add_edge(parent_id, message_id)

In [ ]:
import matplotlib.pyplot as plt

# Visualize a small subgraph (e.g., first 10 nodes)
subgraph = dialogue_graph.subgraph(list(dialogue_graph.nodes)[:10])
nx.draw(subgraph, with_labels=True, node_color="lightblue", edge_color="gray")
plt.show()

In [33]:
from transformers import BertTokenizer, BertModel
import torch

# Load pre-trained BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertModel.from_pretrained("bert-base-uncased")

In [35]:

def get_bert_embeddings(text):
    # Tokenize input text
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)

    # Generate BERT embeddings
    with torch.no_grad():
        outputs = model(**inputs)
    # Use the [CLS] token embedding as the sentence representation
    cls_embedding = outputs.last_hidden_state[:, 0, :].squeeze(0)
    print("==")
    return cls_embedding

# Apply BERT encoding to all messages
bert = pd.read_csv("combined_features.csv")
df_train["bert_embedding"] = bert["bert_embedding"]

In [ ]:
print(df_train)

In [ ]:
import pandas as pd
df_train = pd.read_csv("combined_features.csv")

In [ ]:
import networkx as nx

# Initialize a directed graph
dialogue_graph = nx.DiGraph()

# Add nodes and edges
for idx, row in df_train.iterrows():
    message_id = row["message_id"]
    parent_id = row["parent_id"]
    text = row["text"]
    role = row["role"]

    # Add node with attributes
    dialogue_graph.add_node(message_id, text=text, role=role)

    # Add edge if parent_id exists
    if parent_id:
        dialogue_graph.add_edge(parent_id, message_id)

In [37]:
import re

# Extract sentence-level features
def extract_style_features(text):
    # Sentence length (number of words)
    words = text.split()
    sentence_length = len(words)

    # Punctuation usage
    question_marks = text.count("?")
    exclamation_marks = text.count("!")

    # Vocabulary richness (unique words)
    unique_words = len(set(words))
    vocabulary_richness = unique_words / len(words) if len(words) > 0 else 0

    return {
        "sentence_length": sentence_length,
        "question_marks": question_marks,
        "exclamation_marks": exclamation_marks,
        "vocabulary_richness": vocabulary_richness,
    }

# Apply to all messages
df_train["style_features"] = df_train["text"].apply(extract_style_features)

In [ ]:
import torch

def extract_style_features(row):
    style_dict = row["style_features"]
    return torch.tensor(
        [
            style_dict["sentence_length"],
            style_dict["question_marks"],
            style_dict["exclamation_marks"],
            style_dict["vocabulary_richness"],
        ],
        dtype=torch.float,
    )

# Apply to all rows
df_train["style_features_tensor"] = df_train.apply(extract_style_features, axis=1)

def combine_features(row):
    # BERT embedding (already a tensor)
    bert_embedding = row["bert_embedding"]

    # Style features (already a tensor)
    style_features = row["style_features_tensor"]

    # Graph-based features (e.g., node degree, shortest path length)
    node_degree = dialogue_graph.degree[row["message_id"]]
    shortest_path_length = (
        nx.shortest_path_length(dialogue_graph, row["parent_id"], row["message_id"])
        if row["parent_id"]
        else 0
    )
    graph_features = torch.tensor([node_degree, shortest_path_length], dtype=torch.float)

    # Ensure all tensors are 1-dimensional
    bert_embedding = bert_embedding.squeeze()  # Remove extra dimensions if any
    style_features = style_features.squeeze()  # Remove extra dimensions if any
    graph_features = graph_features.squeeze()  # Remove extra dimensions if any

    # Concatenate all features
    combined_features = torch.cat([bert_embedding, style_features, graph_features], dim=0)
    return combined_features

# Apply to all rows
# Apply to all rows and store results in a list
combined_features_list = []
for _, row in df_train.iterrows():
    combined_features = combine_features(row)
    combined_features_list.append(combined_features)

print(combined_features_list[0])

# Assign the list to the DataFrame
# df_train["combined_features"] = combined_features_list

In [30]:
# df_train["combined_features"] = df_train.apply(combine_features, axis=1)
#
# # Convert combined features to a string representation for CSV storage
# df_train["combined_features"] = df_train["combined_features"].apply(lambda x: ",".join(map(str, x)))

# Save to CSV
df_train.to_csv("combined_features.csv", index=False)

In [11]:
pip install ast

  Using cached AST-0.0.2.tar.gz (19 kB)
  Installing build dependencies ... one
  Getting requirements to build wheel ... error
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [24 lines of output]
      Traceback (most recent call last):
        File "/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/pip/_vendor/pyproject_hooks/_in_process/_in_process.py", line 389, in <module>
          main()
          ~~~~^^
        File "/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/pip/_vendor/pyproject_hooks/_in_process/_in_process.py", line 373, in main
          json_out["return_val"] = hook(**hook_input["kwargs"])
                                   ~~~~^^^^^^^^^^^^^^^^^^^^^^^^
        File "/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/pip/_vendor/pyproject_hooks/_in_process/_in_process.py", line 143, in get_requires

In [12]:
import pandas as pd
df_train = pd.read_csv("combined_features.csv")

In [13]:
import networkx as nx

# Initialize a directed graph
dialogue_graph = nx.DiGraph()

# Add nodes and edges
for idx, row in df_train.iterrows():
    message_id = row["message_id"]
    parent_id = row["parent_id"]
    text = row["text"]
    role = row["role"]

    # Add node with attributes
    dialogue_graph.add_node(message_id, text=text, role=role)

    # Add edge if parent_id exists
    if parent_id:
        dialogue_graph.add_edge(parent_id, message_id)

In [14]:
print(df_train["combined_features"].apply(type).value_counts())

combined_features
<class 'str'>    39283
Name: count, dtype: int64


In [15]:
import torch
import ast

def string_to_tensor(tensor_string):
    # Remove the "tensor(" prefix and ")" suffix
    tensor_string = tensor_string.replace("tensor(", "").replace(")", "")

    # Convert the string to a list of floats
    tensor_list = ast.literal_eval(tensor_string)

    # Convert the list to a PyTorch tensor
    return torch.tensor(tensor_list, dtype=torch.float)

# Apply the conversion to the "combined_features" column
df_train["combined_features"] = df_train["combined_features"].apply(string_to_tensor)

# Verify the conversion
print(df_train["combined_features"].apply(type).value_counts())

combined_features
<class 'torch.Tensor'>    39283
Name: count, dtype: int64


In [16]:
combined_features_list=[]
for _, row in df_train.iterrows():
    combined_features_list.append(row["combined_features"])


In [17]:
from torch_geometric.data import Data

# Create a mapping from string node IDs to integer indices
node_id_to_index = {node_id: idx for idx, node_id in enumerate(dialogue_graph.nodes)}
# Convert edges to integer indices
edges = [(node_id_to_index[src], node_id_to_index[dst]) for src, dst in dialogue_graph.edges]

# Example output: [(0, 1), (2, 3), ...]
print(edges)

# Example output: {'msg_0_human': 0, 'msg_0_gpt': 1, 'msg_1_human': 2, ...}
print(node_id_to_index)

# Create edge index
edge_index = torch.tensor(edges, dtype=torch.long).t().contiguous()
# Create node features tensor
node_features = torch.stack(combined_features_list)

# Create PyTorch Geometric data object
graph_data = Data(x=node_features, edge_index=edge_index)

[(0, 2), (0, 4), (0, 6), (1, 0), (1, 13), (1, 26), (1, 39), (1, 51), (1, 63), (1, 80), (1, 85), (1, 97), (1, 101), (1, 113), (1, 126), (1, 138), (1, 152), (1, 164), (1, 177), (1, 191), (1, 203), (1, 207), (1, 219), (1, 231), (1, 243), (1, 255), (1, 268), (1, 281), (1, 293), (1, 305), (1, 317), (1, 329), (1, 341), (1, 353), (1, 366), (1, 370), (1, 374), (1, 378), (1, 391), (1, 403), (1, 415), (1, 428), (1, 440), (1, 452), (1, 465), (1, 477), (1, 489), (1, 501), (1, 513), (1, 525), (1, 537), (1, 551), (1, 564), (1, 576), (1, 588), (1, 600), (1, 612), (1, 624), (1, 636), (1, 648), (1, 660), (1, 673), (1, 680), (1, 692), (1, 704), (1, 708), (1, 720), (1, 732), (1, 737), (1, 750), (1, 762), (1, 774), (1, 787), (1, 799), (1, 811), (1, 815), (1, 829), (1, 833), (1, 846), (1, 858), (1, 871), (1, 883), (1, 895), (1, 907), (1, 920), (1, 932), (1, 944), (1, 956), (1, 968), (1, 980), (1, 993), (1, 1006), (1, 1018), (1, 1031), (1, 1043), (1, 1056), (1, 1068), (1, 1080), (1, 1092), (1, 1106), (1, 11

In [18]:
num_nodes = graph_data.x.size(0)
print(f"Number of nodes: {num_nodes}")

Number of nodes: 39283


In [19]:
max_index = torch.max(graph_data.edge_index).item()
print(f"Maximum node index in edge_index: {max_index}")

Maximum node index in edge_index: 39378


In [20]:
# Filter out invalid edges
valid_mask = (graph_data.edge_index[0] < num_nodes) & (graph_data.edge_index[1] < num_nodes)
graph_data.edge_index = graph_data.edge_index[:, valid_mask]

# Update the number of edges
num_edges = graph_data.edge_index.size(1)
print(f"Number of edges after filtering: {num_edges}")

Number of edges after filtering: 39187


In [21]:
# Check the number of nodes and maximum node index
num_nodes = graph_data.x.size(0)
max_index = torch.max(graph_data.edge_index).item()
print(f"Number of nodes: {num_nodes}")
print(f"Maximum node index in edge_index: {max_index}")

# Ensure max_index < num_nodes
assert max_index < num_nodes, "Invalid node indices in edge_index!"

Number of nodes: 39283
Maximum node index in edge_index: 39282


In [22]:
from torch_geometric.utils import coalesce

# Reindex the nodes in edge_index
unique_nodes, edge_index = torch.unique(graph_data.edge_index, return_inverse=True)
edge_index = edge_index.reshape(2, -1)

# Update the number of nodes
num_nodes = unique_nodes.size(0)
print(f"Number of nodes after reindexing: {num_nodes}")

# Update graph_data
graph_data.edge_index = edge_index
graph_data.x = graph_data.x[unique_nodes]  # Reindex node features if necessary

Number of nodes after reindexing: 39283


In [23]:
from torch_geometric.nn import GCNConv
import torch.nn.functional as F

class GCN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)  # First GCN layer
        self.conv2 = GCNConv(hidden_dim, output_dim)  # Second GCN layer

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)  # First convolution
        x = F.relu(x)  # Apply ReLU activation
        x = self.conv2(x, edge_index)  # Second convolution
        return x  # Output node embeddings

In [24]:
# Input dimension: Size of the combined feature vector for each node
input_dim = graph_data.x.size(1)

# Hidden dimension: Size of the hidden layer (can be tuned)
hidden_dim = 128

# Output dimension: Size of the final node embeddings (can be tuned)
output_dim = 64

# Initialize the GCN model
model = GCN(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim)

In [25]:
def compute_loss(out):
    # Reconstruct the adjacency matrix
    adj_reconstructed = torch.sigmoid(torch.mm(out, out.t()))

    # Compute reconstruction loss
    loss = F.binary_cross_entropy(adj_reconstructed, torch.eye(graph_data.x.size(0), device=device))
    return loss

In [71]:
import torch
import torch.optim as optim
from torch_geometric.nn import GCNConv
import torch.nn.functional as F

# Define the GCN model
class GCN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)  # First GCN layer
        self.conv2 = GCNConv(hidden_dim, output_dim)  # Second GCN layer

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)  # First convolution
        x = F.relu(x)  # Apply ReLU activation
        x = self.conv2(x, edge_index)  # Second convolution
        return x  # Output node embeddings

# Input dimension: Size of the combined feature vector for each node
input_dim = graph_data.x.size(1)

# Hidden dimension: Size of the hidden layer (can be tuned)
hidden_dim = 64  # Reduced from 128 to save memory and computation

# Output dimension: Size of the final node embeddings (can be tuned)
output_dim = 32  # Reduced from 64 to save memory and computation

# Initialize the GCN model
model = GCN(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim)

# Move model and data to GPU (if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
graph_data = graph_data.to(device)

# Initialize the optimizer with weight decay (L2 regularization)
optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

# Mixed precision training (if GPU is available)
scaler = torch.cuda.amp.GradScaler(enabled=device.type == "cuda")

# Training loop
model.train()
for epoch in range(10):  # Train for 10 epochs
    optimizer.zero_grad()  # Reset gradients

    # Forward pass with mixed precision
    with torch.cuda.amp.autocast(enabled=device.type == "cuda"):
        out = model(graph_data.x, graph_data.edge_index)  # Forward pass
        loss = compute_loss(out)  # Compute loss

    # Backward pass with gradient scaling
    scaler.scale(loss).backward()  # Scale loss and compute gradients
    scaler.step(optimizer)  # Update model parameters
    scaler.update()  # Update scaler

    print(f"Epoch {epoch + 1}, Loss: {loss.item()}")  # Print loss

# Loss function for graph reconstruction
def compute_loss(out):
    # Reconstruct the adjacency matrix
    adj_reconstructed = torch.sigmoid(torch.mm(out, out.t()))

    # Compute reconstruction loss
    loss = F.binary_cross_entropy(adj_reconstructed, torch.eye(graph_data.x.size(0), device=device))
    return loss

Epoch 1, Loss: 98.96003723144531
Epoch 2, Loss: 98.58274841308594
Epoch 3, Loss: 70.49391174316406
Epoch 4, Loss: 62.91231918334961
Epoch 5, Loss: 56.492408752441406
Epoch 6, Loss: 48.47126007080078
Epoch 7, Loss: 51.87403869628906
Epoch 8, Loss: 50.60769271850586
Epoch 9, Loss: 49.62184143066406
Epoch 10, Loss: 49.938499450683594


In [28]:
import torch
import torch.optim as optim
from torch_geometric.nn import GCNConv
import torch.nn.functional as F

# Define the GCN model
class GCN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)  # First GCN layer
        self.conv2 = GCNConv(hidden_dim, output_dim)  # Second GCN layer

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)  # First convolution
        x = F.relu(x)  # Apply ReLU activation
        x = self.conv2(x, edge_index)  # Second convolution
        return x  # Output node embeddings

# Input dimension: Size of the combined feature vector for each node
input_dim = graph_data.x.size(1)

# Hidden dimension: Size of the hidden layer (can be tuned)
hidden_dim = 128  # Increased hidden dimension for better learning

# Output dimension: Size of the final node embeddings (can be tuned)
output_dim = 64  # Increased output dimension for better representation

# Initialize the GCN model
model = GCN(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim)

# Move model and data to GPU (if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
graph_data = graph_data.to(device)

# Normalize input features (to improve training stability)
graph_data.x = (graph_data.x - graph_data.x.mean(dim=0)) / (graph_data.x.std(dim=0) + 1e-8)

# Initialize the optimizer with weight decay (L2 regularization)
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=5e-4)  # Reduced learning rate

# Mixed precision training (if GPU is available)
scaler = torch.cuda.amp.GradScaler(enabled=device.type == "cuda")

# Loss function for graph reconstruction
def compute_loss(out):
    # Reconstruct the adjacency matrix
    adj_reconstructed = torch.sigmoid(torch.mm(out, out.t()))

    # Use the actual adjacency matrix as the target (if available)
    adj_matrix = torch.sparse_coo_tensor(
        graph_data.edge_index,
        torch.ones(graph_data.edge_index.size(1)),
        size=(graph_data.num_nodes, graph_data.num_nodes),
    ).to_dense().to(device)

    # Compute reconstruction loss
    loss = F.binary_cross_entropy(adj_reconstructed, adj_matrix)
    return loss

# Training loop
model.train()
for epoch in range(30):  # Increased number of epochs
    optimizer.zero_grad()  # Reset gradients

    # Forward pass with mixed precision
    with torch.cuda.amp.autocast(enabled=device.type == "cuda"):
        out = model(graph_data.x, graph_data.edge_index)  # Forward pass
        loss = compute_loss(out)  # Compute loss

    # Backward pass with gradient scaling
    scaler.scale(loss).backward()  # Scale loss and compute gradients
    scaler.step(optimizer)  # Update model parameters,
    scaler.update()  # Update scaler

    print(f"Epoch {epoch + 1}, Loss: {loss.item()}")  # Print loss

/var/folders/rx/krmgq4_535qbq740df6ybxk00000gn/T/ipykernel_32776/2268427952.py:43: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=device.type == "cuda")
/var/folders/rx/krmgq4_535qbq740df6ybxk00000gn/T/ipykernel_32776/2268427952.py:67: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=device.type == "cuda"):


Epoch 1, Loss: 45.827938079833984
Epoch 2, Loss: 23.915206909179688
Epoch 3, Loss: 15.761406898498535
Epoch 4, Loss: 11.886300086975098
Epoch 5, Loss: 9.237792015075684
Epoch 6, Loss: 7.043344497680664
Epoch 7, Loss: 5.246325969696045
Epoch 8, Loss: 4.205758094787598
Epoch 9, Loss: 3.6158456802368164
Epoch 10, Loss: 3.181508779525757
Epoch 11, Loss: 2.786644220352173
Epoch 12, Loss: 2.0075294971466064
Epoch 13, Loss: 1.634318470954895
Epoch 14, Loss: 1.481131672859192
Epoch 15, Loss: 1.3875105381011963
Epoch 16, Loss: 1.3122241497039795
Epoch 17, Loss: 1.248957872390747
Epoch 18, Loss: 1.1963471174240112
Epoch 19, Loss: 1.1457992792129517
Epoch 20, Loss: 1.095091462135315
Epoch 21, Loss: 1.0512057542800903
Epoch 22, Loss: 1.019039273262024
Epoch 23, Loss: 0.9992900490760803
Epoch 24, Loss: 0.9883530735969543
Epoch 25, Loss: 0.9800792932510376
Epoch 26, Loss: 0.9703640341758728
Epoch 27, Loss: 0.95786052942276
Epoch 28, Loss: 0.9433237314224243
Epoch 29, Loss: 0.9274409413337708
Epoch 3

In [29]:
model.eval()

with torch.no_grad():
    node_embeddings = model(graph_data.x,graph_data.edge_index)

print("Node embedding shape : ", node_embeddings.shape)

Node embedding shape :  torch.Size([39283, 64])
